In [99]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [100]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-pro')

In [101]:
# tool creation
from langchain_core.tools import InjectedToolArg
from typing import Annotated


@tool
def get_conversion_factor(base_currency: str,target_currency: str) -> float:
    """ 
    This is function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/b7723a06c0cc45d6d8c1c4f5/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_val: int,conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """ 
    Given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_val * conversion_rate

In [102]:
get_conversion_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756771201,
 'time_last_update_utc': 'Tue, 02 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1756857601,
 'time_next_update_utc': 'Wed, 03 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.1343}

In [103]:
convert.invoke({"base_currency_val": 10, "conversion_rate": 85.1343})

851.343

In [104]:
# tool binding

llm_with_tool = model.bind_tools([get_conversion_factor, convert])

In [114]:
messages = [HumanMessage("What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr")]

In [115]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [107]:
ai_message = llm_with_tool.invoke(messages)

In [108]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '7c82c697-c3c2-4eab-9128-cc8104dc86cd',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_val': 10.0},
  'id': 'f82c0a3d-40ed-40ea-a320-a8a3d2402bea',
  'type': 'tool_call'}]

In [116]:
messages.append(ai_message)

In [117]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_val": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--bbedc290-d8a6-4552-b195-0f1ffe519383-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '7c82c697-c3c2-4eab-9128-cc8104dc86cd', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_val': 10.0, 'conversion_rate': 88.1343}, 'id': 'f82c0a3d-40ed-40ea-a320-a8a3d2402bea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 44, 'total_tokens': 783, 'input_token_details': {'cache_read': 0}})]

In [118]:
import json
for tool_call in ai_message.tool_calls:
    # execute the first tool
    if tool_call["name"] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        
        # fetch conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        
        # append this tool message to message list
        messages.append(tool_message1)
    # execute the second tool using the conversion rate from tool 1
    if tool_call["name"] == 'convert':
        # fetch the current args
        tool_call['args']['conversion_rate'] = conversion_rate

        tool_message2 = convert.invoke(tool_call)

        messages.append(tool_message2)

In [119]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_val": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--bbedc290-d8a6-4552-b195-0f1ffe519383-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '7c82c697-c3c2-4eab-9128-cc8104dc86cd', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_val': 10.0, 'conversion_rate': 88.1343}, 'id': 'f82c0a3d-40ed-40ea-a320-a8a3d2402bea', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 44, 'total_tokens': 783, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.ex

In [121]:
llm_with_tool.invoke(messages).content

['881.343',
 "The user wants to know the conversion factor between USD and INR and then use that to convert 10 USD to INR.\n\n1. **`get_conversion_factor`**: This function can be used to find the conversion factor. The `base_currency` is 'USD' and the `target_currency` is 'INR'.\n\n2. **`convert`**: This function can be used to convert the currency. The `base_currency_val` is 10. This function requires a `conversion_rate`.\n\n3. **Pipelining**: The output of the `get_conversion_factor` function (specifically the `conversion_rate`) can be used as the input for the `convert` function.\n\nTherefore, the `get_conversion_factor` tool is called with `base_currency` as `USD` and `target_currency` as `INR`. The result of this is then used to call the `convert` tool with `base_currency_val` as `10`. The final output is the result of the `convert` function."]